In [7]:
from PIL import Image
import os
import json
import shutil

In [8]:
def crop_image_to_aspect(file_path, aspect_ratio, overwrite=False):
  """
  Crop a single image file to the specified aspect ratio (width / height).
  Skips if the image is too short or already fits the ratio.

  Parameters:
      file_path (str): Absolute path to the image file.
      aspect_ratio (float): Desired aspect ratio (width / height), e.g., 4/3.
      overwrite (bool): If True, overwrite the original file. Otherwise, create a new file
                        with '_processed' appended before the extension.
  """
  if not file_path.lower().endswith(('.png', '.jpg', '.jpeg')):
    print(f"Skipping {file_path}: not a supported image type")
    return

  filename = os.path.basename(file_path)
  name, ext = os.path.splitext(filename)

  img = Image.open(file_path)
  width, height = img.size

  # Calculate new height for the given aspect ratio
  new_height = int(width / aspect_ratio)

  # Skip if image is too short or already fits the ratio
  if new_height >= height:
    print(f"Skipping {filename}: cannot crop ({width}x{height}) to {aspect_ratio:.2f}")
    return

  img_cropped = img.crop((0, 0, width, new_height))

  # Determine output path
  if overwrite:
    output_path = file_path
  else:
    output_path = os.path.join(os.path.dirname(file_path), f"{name}_processed{ext}")

  img_cropped.save(output_path)
  print(f"Processed {filename}: {width}x{height} → {width}x{new_height} (saved to {output_path})")

In [9]:
def resize_image_by_width(file_path, width, overwrite=False):
  """
  Resize a single image to the specified width while preserving aspect ratio.
  Skips if the image is already smaller than or equal to the target width.

  Parameters:
      file_path (str): Absolute path to the image file.
      width (int): Target width in pixels.
      overwrite (bool): If True, overwrite the original file. Otherwise, save as a new file
                        with '_processed' appended to the filename.
  """
  if not file_path.lower().endswith(('.png', '.jpg', '.jpeg')):
    print(f"Skipping {file_path}: not a supported image type")
    return

  filename = os.path.basename(file_path)
  name, ext = os.path.splitext(filename)

  img = Image.open(file_path)
  orig_width, orig_height = img.size

  if orig_width <= width:
    print(f"Skipping {filename}: already ≤ target width ({orig_width}px)")
    return

  # Preserve aspect ratio
  new_height = int((width / orig_width) * orig_height)
  img_resized = img.resize((width, new_height), Image.LANCZOS)

  # Determine output path
  if overwrite:
    output_path = file_path
  else:
    output_path = os.path.join(os.path.dirname(file_path), f"{name}_processed{ext}")

  img_resized.save(output_path, optimize=True, quality=85)

  # File sizes in KB
  orig_size_kb = os.path.getsize(file_path) / 1024
  new_size_kb = os.path.getsize(output_path) / 1024

  print(f"Processed {filename}: {orig_width}x{orig_height} → {width}x{new_height}, "
        f"{orig_size_kb:.1f}KB → {new_size_kb:.1f}KB")

In [10]:
def process_all_project_screenshots():
  """
  Process all project screenshots: copy raw_screenshot.jpg → screenshot.jpg,
  then crop to 4/3 and resize to 400px width. The raw_screenshot.jpg file will remain untouched.
  """
  json_file = "projects_data.json"
  if not os.path.isfile(json_file):
    print(f"Error: {json_file} not found in the current folder")
    return

  with open(json_file, 'r', encoding='utf-8') as f:
    projects = json.load(f)

  for project in projects:
    slug = project.get("slug")
    project_folder = os.path.join(os.getcwd(), slug)
    raw_path = os.path.join(project_folder, "raw_screenshot.jpg")
    screenshot_path = os.path.join(project_folder, "screenshot.jpg")

    if not os.path.isfile(raw_path):
      print(f"Skipping {slug}: raw_screenshot.jpg not found")
      continue

    # Step 1: Copy raw_screenshot → screenshot
    shutil.copy2(raw_path, screenshot_path)
    print(f"Copied raw_screenshot.jpg → screenshot.jpg for {slug}")

    # Step 2: Crop and resize
    print(f"Processing {slug} screenshot...")
    crop_image_to_aspect(screenshot_path, aspect_ratio=4/3, overwrite=True)
    resize_image_by_width(screenshot_path, width=400, overwrite=True)

  print("\n✅ All project screenshots processed successfully!")

In [11]:
process_all_project_screenshots()

Skipping array-methods: raw_screenshot.jpg not found
Skipping calculator: raw_screenshot.jpg not found
Skipping cgpa-calculator: raw_screenshot.jpg not found
Skipping cgpa-calculator-2: raw_screenshot.jpg not found
Skipping flexbox-demo: raw_screenshot.jpg not found
Skipping image-search-app: raw_screenshot.jpg not found
Skipping image-search-app-2: raw_screenshot.jpg not found
Skipping rem-vs-px: raw_screenshot.jpg not found
Skipping testimonials: raw_screenshot.jpg not found
Skipping wa-layout: raw_screenshot.jpg not found

✅ All project screenshots processed successfully!
